In [1]:
import sys
import numpy as np
import MDAnalysis as mda
import mbuild as mb
import textwrap
# import inhouse functions
import systop_utils

ModuleNotFoundError: No module named 'systop_utils'

In [3]:
path = '/Users/joedinski/work/Projects/FAIRmat/GRO_DEV/MD_Overview/FSFG/'
sys_nm = 'nvt'
universe = mda.Universe(path+sys_nm+'.tpr', path+sys_nm+'.gro') # use the input coordinate file for positions

In [4]:
# universe.segments.n_segments
# universe.segments.segids
# universe.segments.segindices
#universe.segments.segments[0].atoms
universe.atoms.segids
universe.atoms.segindices
#universe.atoms.n_segments
# universe.atoms.molnums
# universe.atoms.moltypes
universe.segments[0].residues[0].atoms
universe.segments[1].residues[0].atoms
#universe.atoms.fragments

<AtomGroup with 4 atoms>

In [5]:
# parse atom groups (segments and residues)
# we only create atomsgroup in the initial system
system_info = []
for i_segment, segment in enumerate(universe.segments):
    system_info.append({})
    #sec_segment = sec_run.system[0].m_create(AtomsGroup)
    system_info[i_segment]['label'] = segment.segid
    system_info[i_segment]['index'] = int(segment.segindex)
    system_info[i_segment]['atom_indices'] = [atom.index for atom in segment.atoms]
    system_info[i_segment]['residues'] = []
    for i_residue, residue in enumerate(segment.residues):
        system_info[i_segment]['residues'].append({})
        # sec_residue = sec_segment.m_create(AtomsGroup)
        system_info[i_segment]['residues'][i_residue]['label'] = residue.resname
        system_info[i_segment]['residues'][i_residue]['type'] = residue.moltype
        system_info[i_segment]['residues'][i_residue]['index'] = int(residue.resindex)
        system_info[i_segment]['residues'][i_residue]['n_atoms'] = len(residue.atoms)
        system_info[i_segment]['residues'][i_residue]['atom_indices'] = [atom.index for atom in residue.atoms]

In [13]:
system_info[0]

{'label': 'seg_0_Protein',
 'index': 0,
 'atom_indices': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  15

In [20]:
# Construct System Topology
# mol / fragments
# segments
# residues / monomers
# atoms
system = systop_utils.Compound() # store the topology in the mbuild class "Compound"

In [21]:
# first get the atom attributes from the MDAnalysis universe
# nb - moltypes, molnums, elements, and names are defined according to the information available
# moltypes <--> fragtypes
# molnums <--> fragnums
# elements <--> types
# names <--> types
system.get_atom_attributes(universe)
print(system.atoms.moltypes)
print(system.atoms.molnums)
print(system.atoms.resnames)
print(system.atoms.resids)
print(system.atoms.elements)
print(system.atoms.names)

['Protein' 'Protein' 'Protein' ... 'SOL' 'SOL' 'SOL']
[    0     0     0 ... 11247 11247 11247]
['SER' 'SER' 'SER' ... 'SOL' 'SOL' 'SOL']
[    1     1     1 ... 11262 11262 11262]
['N' 'H' 'H' ... 'H' 'H' '']
['N' 'H1' 'H2' ... 'HW2' 'HW3' 'MW4']


In [22]:
# use the atom attributes to construct the basic hierarchical structure molecules-->segments-->residues-->atoms
system.construct_topology()

In [23]:
# add the bonds to the topology structure using the MDAnalysis universe
# nb - the mbuild Compound class stores the bonds as a graph (similar to a networkx object)
#    - it is quite expensive to add the bonds one by one...not even sure what the utility of this is, we might just add
#       a reference to the bond list in the force field (or vica versa).
system.add_bonds_from_universe(universe)

KeyboardInterrupt: 

In [24]:
# we can add a representative structure to the topology structure
# nb - this could just be a reference to the input/starting configuration
system.add_positions_from_universe(universe)

In [25]:
# Let's illustrate the topology
print('The system is '+str(system))
print('The first molecule is '+str(system.children[0]))
print('The first residue of the first molecule is '+str(system.children[0].children[0]))
print('The first atom in the first residue of the first molecule is '+str(system.children[0].children[0].children[0]))

The system is <Compound 45194 particles, non-periodic, 92 bonds, id: 140236627233936>
The first molecule is <Protein 105 particles, non-periodic, 92 bonds, id: 140236612778384>
The first residue of the first molecule is <SER 13 particles, non-periodic, 12 bonds, id: 140236598983952>
The first atom in the first residue of the first molecule is <N pos=( 30.2700, 17.3300, 41.4400), 0 bonds, id: 140236627232528>


In [26]:
# Let's illustrate the topology
print('The system is '+str(system))
print('The second molecule is '+str(system.children[1]))
#print('The first residue of the second molecule is '+str(system.children[1].children[0]))
print('The first atom in the second molecule is '+str(system.children[1].children[0]))

The system is <Compound 45194 particles, non-periodic, 92 bonds, id: 140236627233936>
The second molecule is <Protein 105 particles, non-periodic, 0 bonds, id: 140236478206800>
The first atom in the second molecule is <SER 13 particles, non-periodic, 0 bonds, id: 140236625265936>


In [20]:
# with the mbuild Compound class, we can easily view the system or subsystems with nglview
system.children[0].children[0]._visualize_nglview()

/Users/joedinski/miniconda3/envs/mosdef37/lib/python3.7/site-packages/mbuild/box.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mins = np.array(mins, dtype=np.float)
/Users/joedinski/miniconda3/envs/mosdef37/lib/python3.7/site-packages/mbuild/box.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  maxs = np.array(maxs, dtype=np.flo

NGLWidget()

In [27]:
# Here we use the atom attributes to gather a summary/overview of the topology
# nb - this could probably also be done directly from the topology data structure...that would probably be more general/robust
system.get_topology_summary()

In [28]:
for moltype in system.molnames:
    print('MOLTYPE = '+moltype+':')
    print('Number of molecules of this type: '+str(system.moltypes[moltype].count))
    if system.moltypes[moltype].n_res == 1:
        print('A dictionary with the counts of each atom element/type for this molecule type: '+str(system.moltypes[moltype].atom_count))
        print('The formula of this molecule type, with respect to the atoms: '+str(system.moltypes[moltype].atom_formula))
    else:
        print('A dictionary with the counts of each residue type within this molecule type: '+str(system.moltypes[moltype].residue_count))
        print('The formula of this molecule type, with respect to residues: '+str(system.moltypes[moltype].residue_formula))
        print('The full sequence of residues for this molecule type: '+str(system.moltypes[moltype].residue_sequence))
        for res in system.moltypes[moltype].resnames:
            print('A dictionary with the counts of each atom element/type for this residue type (and molecule type): '+str(system.moltypes[moltype].restypes[res].atom_count))
            print('The formula of each residue type (for this molecule type), with respect to the atoms: '+str(system.moltypes[moltype].restypes[res].atom_formula))

MOLTYPE = Protein:
Number of molecules of this type: 2
A dictionary with the counts of each residue type within this molecule type: {'GLY': 1, 'PHE': 2, 'SER': 5}
The formula of this molecule type, with respect to residues: GLY(1)PHE(2)SER(5)
The full sequence of residues for this molecule type: SER.SER.PHE.SER.PHE.GLY.SER.SER
A dictionary with the counts of each atom element/type for this residue type (and molecule type): {'C': 2, 'H': 3, 'N': 1, 'O': 1}
The formula of each residue type (for this molecule type), with respect to the atoms: C(2)H(3)N(1)O(1)
A dictionary with the counts of each atom element/type for this residue type (and molecule type): {'C': 9, 'H': 9, 'N': 1, 'O': 1}
The formula of each residue type (for this molecule type), with respect to the atoms: C(9)H(9)N(1)O(1)
A dictionary with the counts of each atom element/type for this residue type (and molecule type): {'C': 3, 'H': 7, 'N': 1, 'O': 2}
The formula of each residue type (for this molecule type), with respect 

In [27]:
system['Protein_chain_A'][0]
system['NA'][0]

<Compound 1 particles, non-periodic, 0 bonds, id: 140295973323216>

In [37]:
system['Protein_chain_A'][0]['GLN']

[<Compound 17 particles, non-periodic, 0 bonds, id: 140296443471824>,
 <Compound 17 particles, non-periodic, 0 bonds, id: 140296444002576>]